Installlation

In [4]:
%%bash
pip install pytorch-transformers

cd /home/ec2-user/SageMaker/pytorch-transformers/examples
pip install -r requirements.txt


#Fixing error with from tensorboardX import SummaryWriter
#TypeError: __new__() got an unexpected keyword argument 'serialized_options'

source activate pytorch_p36
pip install protobuf -U



  Running setup.py bdist_wheel for sacremoses: started
  Running setup.py bdist_wheel for sacremoses: finished with status 'done'
  Stored in directory: /home/ec2-user/.cache/pip/wheels/70/87/56/e40575cca30d12fee8875d523b8878b7aba866a9f03b2fd983
Successfully built sacremoses
Requirement not upgraded as not directly required: setuptools in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from protobuf) (39.1.0)
Requirement not upgraded as not directly required: six>=1.9 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from protobuf) (1.11.0)
  Found existing installation: protobuf 3.5.2
    Uninstalling protobuf-3.5.2:
      Successfully uninstalled protobuf-3.5.2


fastai 1.0.55 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
fastai 1.0.55 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
fastai 1.0.55 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Download and Generate Data suitable with run_glue example

In [5]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'ipa-cognitive-futures'
object_name = 'cf_datasets/TM/raw/nc020-a04_ibli.csv'
file_name = 'nc020-a04_ibli.csv'
semantic_column = 'EN'

s3.download_file(bucket_name, object_name, file_name)

import pandas as pd
nice_df = pd.read_csv(file_name, header=0,encoding='latin1')

#correct columns
retrieved_columns = ['Class','Classification','EN', 'FR']
columns = retrieved_columns + list(nice_df.columns)[4:]

#obtain EN GS
nice_df.columns = columns
nice_df = nice_df[retrieved_columns]
del nice_df['FR']
nice_df.dropna(inplace=True)
nice_df['Class'] = nice_df['Class'].astype(int)
nice_df['Classification'] = nice_df['Classification'].astype(int)
nice_df.reset_index(inplace=True,drop=True)
nice_df.head()

,Class,Classification,EN
0,1,10001,combusting preparations [chemical additives to...
1,1,10002,adhesives for industrial purposes
2,1,10003,"salt for preserving, other than for foodstuffs"
3,1,10004,auxiliary fluids for use with abrasives
4,1,10005,vulcanization accelerators


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train, dev = train_test_split(nice_df, train_size=0.8, random_state=7, stratify=list(nice_df['Class']))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
nice_df.groupby('Class').count().head(10)

,Classification,EN
Class,,
1,752,752
2,132,132
3,272,272
4,110,110
5,525,525
6,506,506
7,613,613
8,300,300
9,832,832


In [9]:
train.groupby('Class').count().head(10)

,Classification,EN
Class,,
1,602,602
2,106,106
3,218,218
4,88,88
5,420,420
6,405,405
7,490,490
8,240,240
9,666,666


In [10]:
dev.groupby('Class').count().head(10)

,Classification,EN
Class,,
1,150,150
2,26,26
3,54,54
4,22,22
5,105,105
6,101,101
7,123,123
8,60,60
9,166,166


In [11]:
dataset_folder = '/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets/gns'
#train.to_csv(dataset_folder + '/train.tsv', sep='\t', index=False, header=False)
#dev.to_csv(dataset_folder + '/dev.tsv', sep='\t', index=False, header=False)

In [18]:
train = pd.read_csv(dataset_folder + '/train.tsv', sep='\t',  header=None)
train.columns = ['Class','Classification','En']
dev = pd.read_csv(dataset_folder + '/dev.tsv', sep='\t',  header=None)
dev.columns = ['Class','Classification','En']
dev.head()


,Class,Classification,En
0,20,200235,stools
1,3,30082,diamantine [abrasive]
2,25,250095,cuffs
3,22,220071,tents*
4,11,110135,purification installations for sewage


### Run Glue Training on GnS

In [21]:
%%sh

export GLUE_DIR=/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets
export TASK_NAME=gns

python /home/ec2-user/SageMaker/pytorch-transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name $TASK_NAME \
    --do_train \
    --do_eval \
    --do_lower_case \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 15 \
    --per_gpu_eval_batch_size=512   \
    --per_gpu_train_batch_size=512   \
    --learning_rate 2e-5 \
    --num_train_epochs 30.0 \
    --output_dir $GLUE_DIR/output/$TASK_NAME/ 2>&1 | tee run.log


09/19/2019 14:41:04 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
09/19/2019 14:41:05 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/ec2-user/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
09/19/2019 14:41:05 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "gns",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 45,
  "output_attentions": false,
  "output_hidden_states": false,
  "p

# For Training V2 (bert-large)

In [ ]:
%%sh

export GLUE_DIR=/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets
export TASK_NAME=gns

python /home/ec2-user/SageMaker/pytorch-transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-large-uncased \
    --task_name $TASK_NAME \
    --do_train \
    --do_eval \
    --do_lower_case \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 15 \
    --per_gpu_eval_batch_size=128   \
    --per_gpu_train_batch_size=128   \
    --learning_rate 2e-5 \
    --num_train_epochs 100.0 \
    --output_dir $GLUE_DIR/output/$TASK_NAME/bert-large/ 2>&1 | tee run_bert_large_uncased.log


In [ ]:
%%sh

export GLUE_DIR=/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets
export TASK_NAME=gns

python /home/ec2-user/SageMaker/pytorch-transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-large-uncased \
    --task_name $TASK_NAME \
    --do_eval \
    --do_lower_case \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 15 \
    --per_gpu_eval_batch_size=128   \
    --per_gpu_train_batch_size=128   \
    --learning_rate 2e-5 \
    --num_train_epochs 100.0 \
    --output_dir $GLUE_DIR/output/$TASK_NAME/bert-large/ 2>&1 | tee run_bert_large_uncased.log


# For Training V3 (roberta-large)

In [ ]:
%%sh

export GLUE_DIR=/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets
export TASK_NAME=gns

python /home/ec2-user/SageMaker/pytorch-transformers/examples/run_glue.py \
    --model_type roberta \
    --model_name_or_path roberta-large \
    --task_name $TASK_NAME \
    --do_train \
    --do_eval \
    --do_lower_case \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 15 \
    --per_gpu_eval_batch_size=92   \
    --per_gpu_train_batch_size=92   \
    --learning_rate 2e-5 \
    --num_train_epochs 100.0 \
    --overwrite_output_dir \
    --output_dir $GLUE_DIR/output/$TASK_NAME/roberta-large/ 2>&1 | tee run_roberta_large.log


In [ ]:
# For Training V4 (xlnet-large)

In [ ]:
%%sh

export GLUE_DIR=/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets
export TASK_NAME=gns

python /home/ec2-user/SageMaker/pytorch-transformers/examples/run_glue.py \
    --model_type xlnet \
    --model_name_or_path xlnet-large-cased \
    --task_name $TASK_NAME \
    --do_train \
    --do_eval \
    --do_lower_case \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 15 \
    --per_gpu_eval_batch_size=92   \
    --per_gpu_train_batch_size=92   \
    --learning_rate 2e-5 \
    --num_train_epochs 100.0 \
    --overwrite_output_dir \
    --output_dir $GLUE_DIR/output/$TASK_NAME/xlnet-large-cased/ 2>&1 | tee run_xlnet_large_cased.log
